# SNAP Eligibility Estimation & Penetration Rate Calculation
This part of the repository will be focused on the estimation of eligibility population in the Hennepin County across the tracts (the geographical units upon which the Census Bureau carries out surveys), races and ethics, and time.

The main data sources used in this notebook consists of mainly two parts:
1. The SNAP registration data generously provided by the Hennepin County, which is available in the repository: 'SNAP Summary data for LiveCase fall 2020.xlsx'.
2. The survey data we use to estimate the total eligible people, which is provided by the Census API. You will find a full pipeline to obtain, transform, and adjust the data to arrive the estimations in this notebook.

For the second part, we use in total of 3 data sources, all from census.gov: ACS5, ACS1, and CPS survey data. The reason for refering to different data sources is to make our estimation as accurate as possible. Otherwise the change in penetration rate will only depend on the change in registered people, which is not a good representation for performance and coverage.

**ACS5** is a aggregation of 5 years of ACS1 data, providing detailed estimations on under-poverty line population across tracts and races. We use this data as a baseline. However, since the actual number of eligibility should be changing across each month due to lots of reasons (change in population base and the ratio of population under poverty line).

**ACS1** is the data we use the reflect the change in the population base for each tract and race from year to year.

**CPS** is a monthly timeseries survey data source and is perfect for adjustments to reflect the change in ratio of people under poverty (reflecting monthly fluctuations in the economy).

We will first obtain the data one by one and use them to arrive the estimation of eligibility, and them use it to divide registration data to get the penetration data we care about.

In [1]:
import pandas as pd
import numpy as np
import geopandas as gp
from glob import glob
import urllib
import json
import os
from zipfile import ZipFile
from matplotlib import pyplot as plt
from multiprocessing import Pool

## CPS: Monthly Change in Eligible Rate
(Need to download files, so might take some time)

This section will download around 600 Mb of zip files for the CPS data from 2016 Jan to 2019 Dec. If you are using this notebook for later periods, please change the following lists.

If you wish to skip downloading, please use the CPS output results in the repository: 'output/mn_cps_income_2016_2019.csv'

In [2]:
months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
years = ['19', '18', '17', '16']

In [3]:
if not os.path.exists('Download'):
    os.makedirs('Download') 

In [4]:
url_head = 'https://www2.census.gov/programs-surveys/cps/datasets/20{1}/basic/{0}{1}pub.zip'
download_path = 'Download/{0}{1}pub.zip'
total = len(months) * len(years)
current = 1
for year in years:
    for month in months:
        url = url_head.format(month, year)
        download = download_path.format(month, year)
        print('Downloading {}{}... {}/{}'.format(month, year, current, total))
        if not os.path.exists(download):
            urllib.request.urlretrieve(url, download)
        print('Complete!')
        current += 1

Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!
Complete!


In [6]:
zipfiles = glob('Download/*pub.zip')

for file in zipfiles:
    with ZipFile(file, 'r') as z:
        z.extractall('Download/')

In [7]:
files = glob('Download/*.dat')

The CPS data is stored in an unusual structure where each field takes up a fixed position in each row. Here we only need five fields as shown below. You can find a full list of the avaliable fields in the [data dictionary](https://www2.census.gov/programs-surveys/cps/datasets/2020/basic/2020_Basic_CPS_Public_Use_Record_Layout_plus_IO_Code_list.txt).

In [8]:
variables = {'year':(17,21),
             'month':(15,17),
             'income':(38,40),
             'family_count':(58,60),
             'race':(138,140)}

In [9]:
df = pd.DataFrame()
for file in files:
    tmp = pd.DataFrame([[line[var[0]:var[1]] for var in variables.values()] for line in 
                       open(file, 'r') if line[92:94] == '27'], columns=variables.keys())
    df = pd.concat([df, tmp])

In [10]:
# see disctionary
race_map =  {'1': 'WhiteOnly',
             '2': 'BlackOnly',
             '3': 'AmericanIndian,AlaskanNativeOnly',
             '4': 'AsianOnly',
             '5': 'Hawaiian/PacificIslanderOnly',
             '6': 'White-Black',
             '7': 'White-AI',
             '8': 'White-Asian',
             '9': 'White-HP',
             '10': 'Black-AI',
             '11': 'Black-Asian',
             '12': 'Black-HP',
             '13': 'AI-Asian',
             '14': 'AI-HP',
             '15': 'Asian-HP',
             '16': 'W-B-AI',
             '17': 'W-B-A',
             '18': 'W-B-HP',
             '19': 'W-AI-A',
             '20': 'W-AI-HP',
             '21': 'W-A-HP',
             '22': 'B-AI-A',
             '23': 'W-B-AI-A',
             '24': 'W-AI-A-HP',
             '25': 'Other3RaceCombinations',
             '26': 'Other4and5RaceCombinations',
             '-1': np.nan}

In [11]:
df['race'] = df['race'].str.replace(' ','').map(race_map)

df[['year', 'month', 'income', 'family_count']] =\
df[['year', 'month', 'income', 'family_count']].astype(int)

# exclude the households with unknown income
df = df[df['income']!=-1]

Eligibility criteria for SNAP gross income limits.

See website: https://www.fns.usda.gov/snap/recipient/eligibility

In [13]:
eligible_mapping =  {1: 16596,
                     2: 22416,
                     3: 28236,
                     4: 34068,
                     5: 39888,
                     6: 45708,
                     7: 51540,
                     8: 57360}

The income field in the CPS data only provides an income range for each household (see dictionary). For estimation purpose, we use the mid-point of each range to compare with the SNAP criteria. We also checked the robustness in this treatment and found no significant difference in the final results whether using lower bound, mid-point, or higher bound for estimation.

In [12]:
# Higher Range
# income_mapping ={1: 5000,
#                  2: 7500,
#                  3: 10000,
#                  4: 12500,
#                  5: 15000,
#                  6: 20000,
#                  7: 25000,
#                  8: 30000,
#                  9: 35000,
#                  10: 40000,
#                  11: 50000,
#                  12: 60000,
#                  13: 75000,
#                  14: 100000,
#                  15: 150000,
#                  16: np.inf}
# Lower Range
# income_mapping ={1: 0,
#                  2: 7500,
#                  3: 7500,
#                  4: 10000,
#                  5: 12500,
#                  6: 15000,
#                  7: 20000,
#                  8: 25000,
#                  9: 30000,
#                  10: 35000,
#                  11: 40000,
#                  12: 50000,
#                  13: 60000,
#                  14: 75000,
#                  15: 100000,
#                  16: 150000}
# Middle Range
income_mapping ={1: 5000,
                 2: 6750,
                 3: 8750,
                 4: 11250,
                 5: 13750,
                 6: 17500,
                 7: 22500,
                 8: 27500,
                 9: 32500,
                 10: 37500,
                 11: 45000,
                 12: 55000,
                 13: 67500,
                 14: 87500,
                 15: 125000,
                 16: np.inf}
def is_eligible(family_count, income_level, eligible_mapping, income_mapping):
    if family_count <= 8:
        threshold = eligible_mapping[family_count]
    else:
        threshold = 486 * (family_count-8) + 57360
    income = income_mapping[income_level]
    if income > threshold:
        eligible = False
    else:
        eligible = True
    
    return eligible  

In [14]:
df['eligible'] = df.apply(lambda x: is_eligible(x['family_count'], x['income'], 
                                                eligible_mapping, income_mapping), axis=1)

df['eligible_count'] = df['family_count'] * df['eligible']

df.to_csv('output/mn_cps_income_2016_2019.csv',index=None)

Please Start from here if you would like to skip the downloading part.

In [15]:
df = pd.read_csv('output/mn_cps_income_2016_2019.csv')
cps = df.groupby(['year', 'month'])[['family_count', 'eligible_count']].sum().reset_index()
cps['cps_poverty_ratio'] = cps['eligible_count'] / cps['family_count']

In [16]:
cps.head()

,year,month,family_count,eligible_count,cps_poverty_ratio
0,2016,1,5289,1068,0.201929
1,2016,2,5549,1107,0.199495
2,2016,3,5327,911,0.171016
3,2016,4,5325,1033,0.193991
4,2016,5,5068,1120,0.220994


In [17]:
cps.to_csv('output/mn_cps_eligibility_by_month.csv', index=None)

## ACS1: Yearly Change in Population by Race
The ACS data API takes a very different structure. Please check the [api variables page](https://api.census.gov/data/2018/acs/acs1/subject/variables.html) for example.

In [18]:
df = pd.DataFrame()

for year in range(2016,2020):
    url = 'https://api.census.gov/data/{}/acs/acs1/?get=B03002_001E,B03002_003E,'.format(year)+\
    'B03002_004E,B03002_005E,B03002_006E,B03002_007E,B03002_008E,B03002_009E,B03002_012E&for=county:*'+\
    '&in=state:27'

    col_names = ['Total', 'White Alone, Not Hispanic or Latino', 'Black or African American Alone', 
                 'American Indian and Alaska Native Alone', 'Asian Alone', 
                 'Native Hawaiian and Other Pacific Islander Alone', 'Some Other Race Alone',
                 'Two or More Races', 'Hispanic or Latino', 'state', 'county']

    with urllib.request.urlopen(url) as r:
        tmp = pd.DataFrame(json.loads(r.read())[1:], columns=col_names)
        
    tmp = tmp[tmp['county']=='053']
    tmp = tmp.astype(int)
    tmp['year'] = year
    df = pd.concat([df,tmp])

In [19]:
population = pd.melt(df, 'year', 
                     ['White Alone, Not Hispanic or Latino', 'Black or African American Alone',
                      'American Indian and Alaska Native Alone', 'Asian Alone',
                      'Native Hawaiian and Other Pacific Islander Alone', 'Some Other Race Alone',
                      'Two or More Races', 'Hispanic or Latino'], 
                     'Race_Ethnicity', 'county_population')

In [22]:
Race_Ethnicity_Mapping = {'Black or African American Alone':'Black/African American',
                          'American Indian and Alaska Native Alone':'American Indian or Alaskan Native',
                          'Asian Alone':'Asian/Pacific Islander',
                          'Native Hawaiian and Other Pacific Islander Alone':'Asian/Pacific Islander',
                          'Some Other Race Alone':'Other or Unknown',
                          'Two or More Races':'Other or Unknown',
                          'White Alone, Not Hispanic or Latino':'White',
                          'Hispanic or Latino':'Hispanic or Latino'}
population['Race_Ethnicity'] = population['Race_Ethnicity'].map(Race_Ethnicity_Mapping)
population.head()

,year,Race_Ethnicity,county_population
0,2016,White,855894
1,2017,White,860662
2,2018,White,860981
3,2019,White,863985
4,2016,Black/African American,156239


In [24]:
population.to_csv('output/acs1_population_by_race.csv',index=None)

## Use the Above Data to Adjusting ACS5, the Baseline

There are in total three steps of adjustment.

In [27]:
poverty = pd.read_csv('output/tract_poverty_data_acs5_2018.csv',dtype={'tract':str})
poverty = poverty[poverty['Race_Ethnicity']!='Total Population'][['tract', 'Total', 'Below_Poverty_Count', 'Race_Ethnicity']]
poverty = poverty[poverty['Total']!=0]

In [28]:
poverty['Race_Ethnicity'] = poverty['Race_Ethnicity'].map(Race_Ethnicity_Mapping)

### 1. 100 Percent to 125 Percent Adjustment

Population under poverty line provided by ACS5 are not exactly matching the SNAP criteria of gorss income under 130% of poverty line. So the first step is to adjust it to the 125% of poverty line (best approximation we can get from the data).

In [29]:
url = 'https://api.census.gov/data/2018/acs/acs5/subject?get=NAME,S1701_C01_039E,S1701_C02_001E' +\
      '&for=tract:*&in=state:27&in=county:053'

col_names = ['Name', '125%_poverty', '100%_poverty', 'state','county','tract']

with urllib.request.urlopen(url) as r:
    df = pd.DataFrame(json.loads(r.read())[1:], columns=col_names)

df[['125%_poverty', '100%_poverty']] = df[['125%_poverty', '100%_poverty']].astype(int)

In [30]:
poverty = pd.merge(poverty, df[['tract', '125%_poverty', '100%_poverty']], on='tract')

poverty['below_125%_estimate'] = poverty['Below_Poverty_Count'] * (poverty['125%_poverty']/poverty['100%_poverty'])

## 2. Total Population Adjust by Year (Use ACS1)

In [31]:
population = pd.read_csv('output/acs1_population_by_race.csv')

In [32]:
population['merge'] = 1
poverty['merge'] = 1

In [45]:
poverty_adjust = pd.merge(poverty, population, how='outer', on=['merge','Race_Ethnicity'])

poverty_adjust['acs5_county_population'] = poverty_adjust.groupby(['year','Race_Ethnicity'])['Total'].transform(sum)

poverty_adjust['below_125%_estimate'] = poverty_adjust['below_125%_estimate'] * poverty_adjust['county_population'] / poverty_adjust['acs5_county_population']

## 3. Adjust Poverty Ratio Over Time (use CPS)

In [46]:
acs_poverty_ratio = poverty_adjust['below_125%_estimate'].sum() / poverty_adjust['Total'].sum()

In [47]:
# Overall eligible rate in Hennepin County
# We will try to use CPS data to make it reflect the changes across time
acs_poverty_ratio

0.12157282751665367

In [48]:
poverty_adjust['poverty_ratio'] = poverty_adjust['below_125%_estimate'] / poverty_adjust['Total']

In [53]:
cps = pd.read_csv('output/mn_cps_eligibility_by_month.csv')

In [50]:
cps['merge'] = 1
poverty_adjust['merge'] = 1
poverty_adjust = pd.merge(cps[['year', 'month', 'cps_poverty_ratio', 'merge']], 
                          poverty_adjust[['tract', 'Race_Ethnicity', 'below_125%_estimate', 'poverty_ratio', 'merge']], 
                          on='merge').drop('merge', axis=1)

In [51]:
poverty_adjust['poverty_ratio_adj'] = poverty_adjust['poverty_ratio'] * (poverty_adjust['cps_poverty_ratio'] / acs_poverty_ratio)

poverty_adjust.loc[poverty_adjust['poverty_ratio_adj']>1, 'poverty_ratio_adj'] = 1

## Estimated Eligible

In [52]:
poverty_adjust['Estimated Eligible'] = (poverty_adjust['poverty_ratio_adj'] * poverty_adjust['below_125%_estimate']).round()

In [54]:
estimated_eligible = poverty_adjust[['year','month','tract','Race_Ethnicity','Estimated Eligible']]

estimated_eligible.to_csv('output/estimated_eligible.csv',index=None)

## Combine with Registration Data to Calculate Penetration Rate

In [55]:
snap = pd.read_excel('SNAP Summary data for LiveCase fall 2020.xlsx')

In [56]:
snap['elig_month'] = pd.to_datetime(snap['elig_month'])
snap['tract'] = snap['tract'].astype(str)

snap = snap[snap['tract'].str[2:5]=='053'].rename(columns={'race_ethnicity':'Race_Ethnicity'})

snap['tract'] = snap['tract'].str[-6:]

In [57]:
estimated_eligible['elig_month'] = pd.to_datetime(estimated_eligible['year'].astype(str)+'-'+estimated_eligible['month'].astype(str)+'-1')

<ipython-input-57-a4fde5035445>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  estimated_eligible['elig_month'] = pd.to_datetime(estimated_eligible['year'].astype(str)+'-'+estimated_eligible['month'].astype(str)+'-1')


In [58]:
snap = pd.merge(estimated_eligible[['elig_month', 'tract', 'Race_Ethnicity', 'Estimated Eligible']], 
         snap, how='left', on=['elig_month', 'tract', 'Race_Ethnicity'])

In [59]:
snap['people'] = snap['people'].fillna(0)
snap['coverage'] = snap['people'] / snap['Estimated Eligible']
snap.loc[snap['coverage']==np.inf, 'coverage'] = np.nan
# cap the rate to 100%
snap.loc[snap['coverage']>1, 'coverage'] = 1

snap.to_csv('output/results.csv', index=None)